<a href="https://colab.research.google.com/github/AlexeyBudarov/Titanic.-Kaggle-competition.-Binary_classification/blob/main/Titanic_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data analysis**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import copy

Read and data analysis

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
# df_train.info()
target = df_train['Survived']

In [ ]:
for i in range(len(df_train.columns)):
  print(f"{i} {df_train.columns[i]}  {sorted(set(df_train[df_train.columns[i]].dropna()))}")
  # print(f"{df[df.columns[i]].info()}")
  # print()

In [ ]:
def Split_type_cols(df_train):
  cat_cols = []
  cat_cols_index = []
  num_cols = []
  num_cols_index = []
  for counter, column in enumerate(df_train.columns):
    if df_train[column].dtypes == object:
      cat_cols = np.append(cat_cols, column)
      cat_cols_index = np.append(cat_cols_index, counter)
    elif i != 'Survived':
      num_cols = np.append(num_cols, column)
      num_cols_index = np.append(num_cols_index, counter)
  return cat_cols, cat_cols_index.astype(int), num_cols, num_cols_index.astype(int)

In [ ]:
def Del_columns(data, name):
  new_data = data.drop(name, axis = 1)
  cat_cols_new = np.delete(cat_cols, np.where(cat_cols == name)[0])
  return new_data, cat_cols_new

Function for splitting ticket on serial and number. Add this two columns in our data and remove old column. **Notion - bad idea, prediction has worsened**

In [ ]:
def To_int(x):
    try:
      x = int(x)
    except ValueError:
      x = 1
    return x

In [ ]:
def Ticket_series(x):
  temp = x.split(' ')
  if len(temp) == 1:
    return '0'
  return '_'.join(temp[0:-1])

In [ ]:
df_train['Ticket_Number'] = [To_int(x.split(' ')[-1]) for x in df_train['Ticket']]
df_test['Ticket_Number'] = [To_int(x.split(' ')[-1]) for x in df_test['Ticket']]
# df_train['Ticket_Number'] = [x.split(' ')[-1] for x in df_train['Ticket']]
# df_test['Ticket_Number'] = [x.split(' ')[-1] for x in df_test['Ticket']]
df_train['Ticket_Series'] = [Ticket_series(x) for x in df_train['Ticket']]
df_test['Ticket_Series'] = [Ticket_series(x) for x in df_test['Ticket']]

df_train = df_train.drop('Ticket', axis = 1)
df_test = df_test.drop('Ticket', axis =1 )

In [ ]:
df_train['Name'] = [" ".join([v.strip(",()[].\"'") for v in x.split(" ")]) for x in df_train['Name']]
df_test['Name'] = [" ".join([v.strip(",()[].\"'") for v in x.split(" ")]) for x in df_test['Name']]

## Data for catboost

In [ ]:
df_train_catboost = copy.deepcopy(df_train)
df_test_catboost = copy.deepcopy(df_test)


# df_test_catboost = df_test_catboost.drop('Name', axis = 1)
# df_train_catboost = df_train_catboost.drop('Name', axis = 1)

df_test_catboost = df_test_catboost.drop('PassengerId', axis = 1)
df_train_catboost = df_train_catboost.drop('PassengerId', axis = 1)

df_train_catboost = df_train_catboost.drop('Survived', axis = 1)


cat_cols_cb, cat_cols_cb_index, num_cols_cb, num_cols_cb_index = Split_type_cols(df_train_catboost)
print(f"cat_cols: {cat_cols_cb} - {cat_cols_cb_index}\nnum_cols: {num_cols_cb} - {num_cols_cb_index}")


df_train_catboost[cat_cols_cb] = df_train_catboost[cat_cols_cb].fillna('')
df_test_catboost[cat_cols_cb] = df_test_catboost[cat_cols_cb].fillna('')

df_train_catboost[num_cols_cb] = df_train_catboost[num_cols_cb].fillna(0)
df_test_catboost[num_cols_cb] = df_test_catboost[num_cols_cb].fillna(0)

cat_cols: ['Name' 'Sex' 'Ticket' 'Cabin' 'Embarked'] - [ 2  3  7  9 10]
num_cols: ['PassengerId' 'Pclass' 'Age' 'SibSp' 'Parch' 'Fare'] - [0 1 4 5 6 8]


## General data prepocessing

In [ ]:
df_train['Embarked'] = df_train['Embarked'].replace({'S': 1, 'C': 2, 'Q': 3, np.nan:0})
df_test['Embarked'] = df_test['Embarked'].replace({'S': 1, 'C': 2, 'Q': 3, np.nan:0})
df_train['Sex'] = df_train['Sex'].replace({'female': 1, 'male': 0})
df_test['Sex'] = df_test['Sex'].replace({'female': 1, 'male': 0})

In [ ]:
df_train['Cabin'] = df_train['Cabin'].fillna('0')
df_test['Cabin'] = df_test['Cabin'].fillna('0')
df_train['Cabin'].unique()

Think about how to encode cabins and build graphs

In [ ]:
df_train['Cabin'] = [x if x in df_test['Cabin'].unique() else '0' for x in df_train['Cabin']]
df_train['Ticket'] = [x if x in df_test['Ticket'].unique() else '0' for x in df_train['Ticket']]
df_test['Cabin'] = [x if x in df_train['Cabin'].unique() else '0' for x in df_test['Cabin']]
df_test['Ticket'] = [x if x in df_train['Ticket'].unique() else '0' for x in df_test['Ticket']]

This features **only worsen our prediction on all model**

In [ ]:
df_test['Cabin'] = [x if x in df_train['Cabin'].unique() else '0' for x in df_test['Cabin']]
df_train['Cabin'] = [x if x in df_test['Cabin'].unique() else '0' for x in df_train['Cabin']]
df_test['Name'] = [x if x in df_train['Name'].unique() else '0' for x in df_test['Name']]
df_train['Name'] = [x if x in df_test['Name'].unique() else '0' for x in df_train['Name']]

In [ ]:
df_train['Ticket_Number'] = [x if x in df_test['Ticket_Number'].unique() else 0 for x in df_train['Ticket_Number']]
df_test['Ticket_Number'] = [x if x in df_train['Ticket_Number'].unique() else 0 for x in df_test['Ticket_Number']]

df_test['Ticket_Series'] = [x if x in df_train['Ticket_Series'].unique() else '0' for x in df_test['Ticket_Series']]
df_train['Ticket_Series'] = [x if x in df_test['Ticket_Series'].unique() else '0' for x in df_train['Ticket_Series']]

In [ ]:
# df_train = df_train.drop('Name', axis= 1)
# df_test = df_test.drop('Name', axis= 1)
df_train = df_train.drop('PassengerId', axis= 1)
df_test = df_test.drop('PassengerId', axis= 1)
df_train = df_train.drop('Survived', axis= 1)

Select categorial and numeric features

In [ ]:
cat_cols, cat_cols_index, num_cols, num_cols_index = Split_type_cols(df_train)
target_col = 'Survived'
print(cat_cols, cat_cols_index)
print(num_cols, num_cols_index)
print(target_col)

['Name' 'Cabin' 'Ticket_Series'] [ 1  7 10]
['Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Fare' 'Embarked' 'Ticket_Number'] [0 2 3 4 5 6 8 9]
Survived


*Delete empty strings and any columns. Monitoring result our models*

In [ ]:
df_train[cat_cols] = df_train[cat_cols].fillna('0')
df_train[num_cols] = df_train[num_cols].fillna(0)
df_test[num_cols] = df_test[num_cols].fillna(0)
df_test[cat_cols]= df_test[cat_cols].fillna('0')

Work with categorial features. OneHotEncoder

In [ ]:
df_train.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked', 'Ticket_Number', 'Ticket_Series'],
      dtype='object')

In [ ]:
df_test.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked', 'Ticket_Number', 'Ticket_Series'],
      dtype='object')

## Finish processig. Split data and convert categorial features

In [ ]:
from sklearn.model_selection import train_test_split #
from sklearn.preprocessing import OneHotEncoder    #

In [ ]:
df_train = pd.get_dummies(df_train, columns = cat_cols)     # May be convert to function
df_test = pd.get_dummies(df_test, columns = cat_cols)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train,target, train_size = 0.3, random_state = 42)

# **Testing some models**

Select a metric

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

Search Hyperparameters

In [ ]:
def Get_Hyperparam(param_grid, model, X, y):
  GS = GridSearchCV(model, param_grid, cv = 5)
  GS.fit(X,y)
  return GS.best_params_

In [ ]:
def Norm_features(X_train, X_test, Test_data):
  scaler = StandardScaler()
  X_sc_train = scaler.fit_transform(X_train)
  X_sc_test = scaler.transform(X_test)
  Test_sc_data = scaler.transform(Test_data)
  return X_sc_train, X_sc_test, Test_sc_data

In [ ]:
X_train_scaler, X_test_scaler, df_test_scaler = Norm_features(X_train, X_test, df_test)

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
def Decision_tree(X_train, y_train, X_test, Test_data, param_grid, flag):
  tree = DecisionTreeClassifier(**param_grid)
  tree.fit(X_train, y_train)
  if flag == 'train':
    prediction = tree.predict_proba(X_test)[:,1]
    return prediction
  elif flag == 'test':
    trash_hold = 0.5
    Final_prediction = (tree.predict_proba(Test_data)[:,1] > trash_hold).astype(int)
    return Final_prediction

In [ ]:
tree_param_grid = {'max_depth': np.arange(1,20),
                   'criterion':['gini', 'entropy', 'log_loss']}

Without StandadScaler

In [ ]:
tree_best_param = Get_Hyperparam(tree_param_grid, DecisionTreeClassifier(), X_train, y_train)
print(tree_best_param)

{'criterion': 'gini', 'max_depth': 10}


In [ ]:
train_tree_result = roc_auc_score(y_test, Decision_tree(X_train,y_train, X_test, df_test, tree_best_param, 'train'))
print(train_tree_result)

0.7515099223468507


With StandardScaler

In [ ]:
tree_best_param_sc = Get_Hyperparam(tree_param_grid, DecisionTreeClassifier(), X_train_scaler, y_train)
print(tree_best_param_sc)

{'criterion': 'gini', 'max_depth': 10}


In [ ]:
train_tree_result_sc = roc_auc_score(y_test, Decision_tree(X_train_scaler,y_train, X_test_scaler, df_test_scaler, tree_best_param_sc, 'train'))
print(train_tree_result_sc)

0.7757657463330457


## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def Random_forest(X_train, y_train, X_test, Test_data, param_grid, flag):
  forest = RandomForestClassifier(**param_grid)
  forest.fit(X_train, y_train)
  if flag == 'train':
    prediction = forest.predict_proba(X_test)[:,1]
    return prediction
  elif flag == 'test':
    Final_prediction = forest.predict_proba(Test_data)[:,1]
    trash_hold = 0.5
    Final_prediction = (Final_prediction > trash_hold).astype(int)
    return Final_prediction

In [ ]:
forest_param_grid = {'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']}

Without StandardScaler

In [ ]:
forest_best_param = Get_Hyperparam(forest_param_grid, RandomForestClassifier(), X_train, y_train)
print(forest_best_param)

In [ ]:
forest_train_result = roc_auc_score(y_test, Random_forest(X_train, y_train, X_test, df_test, forest_best_param, 'train'))
print(forest_train_result)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


0.8259760569456427


With StandardScaler

In [ ]:
forest_best_param_sc = Get_Hyperparam(forest_param_grid, RandomForestClassifier(), X_train_scaler, y_train)
print(forest_best_param_sc)

In [ ]:
forest_train_result_sc = roc_auc_score(y_test, Random_forest(X_train_scaler, y_train, X_test_scaler, df_test_scaler, forest_best_param_sc, 'train'))
print(forest_train_result_sc)

0.8475355910267471


## KNearestNeighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def KNN(X_train, y_train, X_test, Test_data, param_grid, flag):
  knn = KNeighborsClassifier(**param_grid)
  knn.fit(X_train, y_train)
  if flag == 'test':
    trash_hold = 0.5
    Final_prediction = (knn.predict_proba(Test_data)[:,1] > trash_hold).astype(int)
    return Final_prediction
  elif flag == 'train':
    prediction = knn.predict_proba(X_test)[:,1]
    return prediction

In [ ]:
knn_param_grid = {'n_neighbors': np.arange(50),
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'weights':['uniform', 'distance']}

Without StandardScaler

In [ ]:
knn_best_param = Get_Hyperparam(knn_param_grid, KNeighborsClassifier(), X_train, y_train)
print(knn_best_param)

In [ ]:
knn_train_result = roc_auc_score(y_test, KNN(X_train, y_train, X_test, df_test, knn_best_param, 'train'))
print(knn_train_result)

0.7206589732528041


With StandardScaler

In [ ]:
knn_best_param_sc = Get_Hyperparam(knn_param_grid, KNeighborsClassifier(), X_train_scaler, y_train)
print(knn_best_param_sc)

In [ ]:
knn_train_result_sc = roc_auc_score(y_test, KNN(X_train_scaler, y_train, X_test_scaler, df_test_scaler, knn_best_param_sc, 'train'))
print(knn_train_result_sc)

0.822470880069025


## CatBoost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
X_cb_train, X_cb_test, y_cb_train, y_cb_test = train_test_split(df_train_catboost.values,
                                                    target.values,
                                                    train_size = 0.7,
                                                    random_state = 74)


In [ ]:
param_grid_ = {'learning_rate': [0.03, 0.1, 0.07],
                'depth': [2, 5, 9],
                'l2_leaf_reg': [3, 5, 9],
                'n_estimators': [200, 300]}

In [ ]:
grid_search = GridSearchCV(CatBoostClassifier(random_seed=42), param_grid= param_grid_, cv=5)
grid_search.fit(X_cb_train, y_cb_train, cat_features= cat_cols_cb_index)
catboost_best_param = grid_search.best_params_
print(catboost_best_param)

In [ ]:
def CatBoost(X_train, y_train,X_test, Test_data, param_grid, cat_cols_index, flag):

  train_pool = Pool(X_train,
                  y_train,
                  cat_features= cat_cols_index)

  test_pool = Pool(X_test,
                 cat_features= cat_cols_index)

  cat = CatBoostClassifier(**param_grid, random_seed=42)
  cat.fit(train_pool)
  if flag == 'train':
    prediction = cat.predict_proba(test_pool)[:,1]
    return prediction
  elif flag == 'test':
    trash_hold = 0.5
    Final_prediction = (cat.predict_proba(Test_data)[:,1] > trash_hold).astype(int)
    return Final_prediction

In [ ]:
catboost_train_result = roc_auc_score(y_cb_test, CatBoost(X_cb_train, y_cb_train, X_cb_test, df_test_catboost, catboost_best_param, cat_cols_cb_index, 'train'))
print(catboost_train_result)

In [ ]:
print(catboost_train_result)

0.8960576640188291


## Stacing

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
def Stacking(X_train, y_train, X_test, Test_data, estimators_, param_grid, flag):
  stack = StackingClassifier(estimators = estimators_, final_estimator = LogisticRegression())
  stack.fit(X_train, y_train)
  if flag == 'test':
    trash_hold = 0.5
    Final_prediction = (stack.predict_proba(Test_data)[:,1] > trash_hold).astype(int)
    return Final_prediction
  elif flag == 'train':
    prediction = stack.predict_proba(X_test)[:,1]
    return prediction

In [ ]:
estimators = [
    ('DecisionTree_1', DecisionTreeClassifier(**tree_best_param)),
    ('DecisionTree_2', DecisionTreeClassifier(criterion = 'gini', max_depth = 12)),
    ('LogReg_2', LogisticRegression()),
    ('KNN', KNeighborsClassifier(**knn_best_param)),
    ('RandomForest_1', RandomForestClassifier(**forest_best_param)),
    ('RandomForest_2', RandomForestClassifier(n_estimators = 154, max_depth = 13, min_samples_split = 3, min_samples_leaf = 2, max_features = 'sqrt')),
    ('LogReg_1', LogisticRegression())

    ]

In [ ]:
stacking_train_result = roc_auc_score(y_test, Stacking(X_train, y_train, X_test, df_test, estimators, knn_best_param, 'train'))
print(stacking_train_result)

In [ ]:
stacking_train_result_sc = roc_auc_score(y_test, Stacking(X_train_scaler, y_train, X_test_scaler, df_test_scaler, estimators, knn_best_param_sc, 'train'))
print(stacking_train_result_sc)

# **Load final final test prediction**

In [ ]:
submission = pd.read_csv('gender_submission.csv')
submission['Survived'] = Stacking(X_train, y_train, X_test, df_test, estimators, knn_best_param, 'test')
submission.to_csv('my_submission_stacking_new.csv', index = False)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor